In [1]:
# The notebook's environment might not recognise local installs of ADRpy, fix that now...
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..\\..\\..\\src")))

<img src="../img/ADRpy_splash.png" width="30%" align="left">

> _"Getting something to fly is easy. What's difficult is getting it to fly well." (Unknown)_

# Aircraft Performance L1 - Scaling Aircraft Concepts

Given a general layout (perhaps one of several candidates at this stage) and some idea of the maximum take-off weight, the next major step of the conceptual design process is the constraint analysis, which will yield the required wing area and thrust - effectively giving 'scale' to the concept.

The constraint analysis establishes the boundaries of the feasible part of the thrust to weight ratio (T/W) versus wing loading (W/S) space. Each element of the mission of the aircraft 'bites out' a chunk from this space and the aeroplane we progress to the subsequent stages of the design process will have to 'live' in the remaining subspace - generally at the lowest thrust/smallest wing point of the remaining feasible chunk.

This approach is effective in identifying wing loadings for which the demand placed on the propulsion system is minimised, as well as which constraints are ambitious or underdeveloped.

## Contents

Arguably one of the most important topics of this notebook:
- [Sea-level static mapping](#Sea-level-static-mapping)

Governing equations for performance constraints based on the conditions at, in, or during:
- [Take-off](#Take-off)
- [Climb](#Climb)
- [Cruise](#Cruise)
- [Sustained turn](#Sustained-turn)
- [Service ceiling](#Service-ceiling)

<!-- _Under construction/consideration. Sorry!_
- ⚠️Instantaneous turn
- ⚠️Descent
- ⚠️Landing -->

## Sea-level static mapping
[Back to Contents ⬆️](#Contents)

The procedure for computing thrust-to-weight ratios required of a concept, for a particular wing loading, is dependent on the concept's experiential lift and drag forces.
These can change drastically depending on the ambient flight conditions, fuel state of the vehicle, embarked payload etc.
Additionally, even first-order approximations of thrust (such as those estiamtes used in conceptual design) will depend on the atmosphere and other dynamic properties of flight (at least, this is the case for air-breathing engines).

The goal of scaling is to have all the design constraints of the aircraft intelligently inform the selection of a suitable propulsion system architecture.
Therefore for example, it is not helpful to directly compare the thrust-to-weight ratios computed at cruise to the thrust-to-weight required for takeoff

$$
\text{nonsensical:} \ \
\frac{T_{\text{climb}}}{W_{\text{climb}}}
\longleftrightarrow
\frac{T_{\text{cruise}}}{W_{\text{cruise}}}
\longleftrightarrow \dots
$$

*Comparing thrust-to-weights as evaluated by the equations of motion between different phases of flight is not helpful.*

Fortunately, ADRpy is here to help.
At the concept sizing stage, it is possible to prescribe the weight of the vehicle at a constraint point as a fraction of the maximum take-off weight $(W_{\text{climb}} / W)$, allowing users to normalise the thrust requirement to a known weight

$$
\text{intriguing:} \ \
\frac{T_{\text{climb}}}{W_{\text{climb}}}
\times
\frac{(W_{\text{climb}} / W)}{1}
= \frac{T_{\text{climb}}}{W}
$$

where $W$ is the MTOW.

Additionally, the thrust (and power) lapses $(T_{\text{climb}} / T)$ and $(P_{\text{climb}} / P)$ due to altitude and forward flight speed above sea-level static can be estimated using models contained witin the `ADRpy.propulsion` module.

$$
\text{delightful:} \ \
\frac{T_{\text{climb}}}{W_{\text{climb}}}
\times
\frac{(W_{\text{climb}} / W)}{(T_{\text{climb}} / T)}
= \frac{T}{W}
$$

where $T$ and $P$ are sea-level static thrust and power outputs, respectively (and often the most easily obtained and quoted performance metrics of an engine).

In ADRpy's `AircraftConcept` class, all T/W and P/W estimates are automatically normalised to sea-level static and MTOW conditions without a need for user intervention.
For more information on the lapse of an engine's available thrust and power, check out the notebook on modelling engine decks.

## Take-off
[Back to Contents ⬆️](#Contents)

### Stall speeds

*Stall speeds* play an important role in early conceptual analysis due to their impact on the take-off, approach, and landing constraints. Generically denoted $V_S$, the stall speed is the minimum airspeed at which the aircraft is able to maintain steady, level flight; in this state the angle of attack is at its critical (or stall) value. 

In an operational context, stall speed is generally expressed in terms of the indicated airspeed (to be precise, at the conceptual design level we take this to be the equivalent airspeed, but, let us use the $V_\mathrm{S (IAS)}$ notation for now), though we can write the lift equation at stall in terms of either IAS or TAS (note the different densities - $\rho_0$ is the sea level standard day ISA density):

$$L = W = \frac{1}{2}\rho_0 V_\mathrm{s (IAS)}^2 SC_\mathrm{Lmax}$$

$$L = W = \frac{1}{2}\rho V_\mathrm{s (TAS)}^2 SC_\mathrm{Lmax}.$$

Of course, we could also write the lift equation in terms of the stall dynamic pressure $q_\mathrm{s}$: $L = W = q_\mathrm{s}SC_\mathrm{Lmax}$ and we will do so later, but, for now, let us express the stall speed from one of the equations above, in order to make its drivers clear:

$$V_\mathrm{s(TAS)} = \sqrt{\frac{2 W/S}{\rho C_\mathrm{Lmax}}}.$$

Obviously the geometry of the aircraft (which determines the $C_\mathrm{Lmax}$) and its wing loading ($W/S$) are the key factors here. For this reason, stall speed related constraints have a massive impact on the cost and the complexity of the aircraft: increasing $C_\mathrm{Lmax}$ (and thus reducing stall speed) is generally achieved through more sophisticated and complicated high lift systems, as well as through more lightly loaded (and thus larger) wings.

In the case of aircraft with variable geometry wings -- such as, wings fitted with high lift devices -- $C_\mathrm{Lmax}$ also varies, so we always need to be specific about which configuration a particular $C_\mathrm{Lmax}$ (and thus, also, $V_\mathrm{s}$) relates to. Certification documents -- such as CS-25 -- use: 

- $V_\mathrm{S0}$ to denote stall speed in the 'everything hanging out' landing configuration and 
- $V_\mathrm{S1}$ to denote stall speed in some other, specified configuration.

### Rotation speed

Another important speed of the take-off is the *rotation speed* $V_\mathrm{R}$. The rotation phase of the take-off begins at the moment when the pilot makes a control input designed to lift the aircraft off the runway. 

The operational reasoning behind the selection of $V_\mathrm{R}$ is that, in case of subsequent engine failure, it should enable the aircraft to reach a take-off safety speed $V_2$ above a specified obstacle height.

For conceptual level take-off analysis the common first order estimate of the rotation speed is $V_\mathrm{R}=1.1V_\mathrm{S1}^\mathrm{TO}$, where $V_\mathrm{S1}^\mathrm{TO}$ is the stall speed in the take-off configuration (this estimate can be written in terms of any type of airspeed).

### Take-off technique

There are some key differences between how very light aircraft (of the order of, say, 10s of kg MTOW) and heavy transports should be treated from a conceptual take-off analysis perspective.

First, the take-off roll technique may be quite different. A radio-controlled aircraft pilot, operating from a grass field, may apply a slight nose-up elevator input some way before $V_R$ to ease the pressure on the wheels, thus reducing their rolling resistance (a similar technique is used on some large aircraft when taking off from contaminated runways). 

Second, very light unmanned aircraft will rotate practically instantaneously, so their lift-off speed $V_\mathrm{L}$ is approximately equal to $V_\mathrm{R}$, while large transport aeroplanes will rotate slowly (2-3 deg/s), lifting off 2.5-4 seconds later, potentially gaining a few more knots in the process (their rotation rates are a compromise between avoiding tail strike and exceeding the speed limits of their tyres).

In general, at the conceptual level, we will make the approximation $V_\mathrm{L}\approx V_\mathrm{R}$.

### Certification requirements 

According to CS 23.51, the corresponding speed $V_R$ must:
- not be less than the greater of 5% above the minimum controllable speed with critical engine inoperative $V_\mathrm{MC}$ and 10% above $V_\mathrm{S1}$ for twin engined aircraft and
- not less than $V_\mathrm{S1}$ for single engined aircraft.

CS 25.107 sets out a similar (though somewhat more complicated) methodology for calculating $V_\mathrm{R}$ for Part 25 aeroplanes. 

Once a concept is arrived at, its performance must be checked against these.

### A simple take-off performance model

#### From Newton's second law to the take-off constraint

To keep the notation as uncluttered as possible, we will, in what follows, consider runway projections of the acceleration, drag, thrust and resultant force without explicitly noting this.

Considering an aircraft on the runway, performing its take-off run, we shall denote the resultant forces acting on the aircraft with $F$. This force imparts the aircraft of mass $m$ an acceleration $a_\mathrm{TO}$:

$$F = m\frac{dV}{dt} = m\,a_\mathrm{TO}$$

From here the acceleration can be written, introducing the aircraft weight $W$, as:

$$a_\mathrm{TO} = \frac{F}{m} = \frac{g\,F}{W}$$

In terms of its projection on the direction of the runway we can write $F$ as the sum of the thrust $T$, the drag $D$ and the landing gear rolling resistance $\mu_\mathrm{R}(W-L)$:

$$a_\mathrm{TO} = \frac{g}{W}\left[T-D-\mu_\mathrm{R}(W-L)\right].$$

In the above equation $g$ and $W$ are constant, but everything else varies during the take-off run. However, at this stage, we may not know enough about our tentative aeroplane to calculate these variations (also, we are after some very fast, first order estimates). Instead, we shall use representative constant values for lift, thrust and drag (denoted by $\overline{L}$, $\overline{T}$ and $\overline{D}$ respectively) and re-write the expression for an assumed constant take-off roll acceleration in these terms:

$$\overline{a_\mathrm{TO}} = \frac{g}{W}\left[\overline{T}-\overline{D}-\mu_\mathrm{R}(W-\overline{L})\right]$$

We are seeking an equation of the form $T/W=f(W/S)$ -- to that end, we re-arrange the above as:

$$\frac{\overline{T}}{W} = \frac{\overline{a_\mathrm{TO}}}{g} + \frac{\overline{D}}{W} + \mu_\mathrm{R}\left(1-\frac{\overline{L}}{W}\right)$$

The acceleration $\overline{a_\mathrm{TO}}$ is that which result in a lift-off speed $V_\mathrm{L}$ at the end of a ground roll distance $d_\mathrm{G}$ is $V^2_\mathrm{L}/2d_\mathrm{G}$ (assuming at this early stage, a uniformly accelerating motion).
The average drag can be written as $\overline{D}=\overline{q}C_D^\mathrm{TO}S$ and the average lift as $\overline{L}=\overline{q}C_L^\mathrm{TO}S$. Assuming that at the end of the take-off roll the aircraft will generate a lift equal to its weight, we can replace the weight in the equation above with $W = q_\mathrm{TO}C_L^\mathrm{TO}S$, so we can rewrite the thrust to weight equation as:

$$\frac{\overline{T}}{W} = \frac{V^2_L}{2d_\mathrm{G}g} + \frac{\overline{q}C_D^\mathrm{TO}S}{q_\mathrm{TO}C_L^\mathrm{TO}S} + \mu_\mathrm{R}\left(1-\frac{\overline{q}C_L^\mathrm{TO}S}{q_\mathrm{TO}C_L^\mathrm{TO}S}\right)$$

$$
\longrightarrow \quad
\frac{\overline{T}}{W}
= \frac{V^2_L}{2d_\mathrm{G}g}
+ \frac{C_D^\mathrm{TO}}{C_L^\mathrm{TO}} \cdot \left(\frac{\overline{q}}{q_\mathrm{TO}}\right)
+ \mu_\mathrm{R} \cdot \left(1-\frac{\overline{q}}{q_\mathrm{TO}}\right)
$$

For aircraft scaling purposes, we are looking for the limiting case in which $\overline{T}/W$ is maximised.
If we neglect the penalty of airspeed on the thrust lapse (see [sea-level static mapping](#Sea-level-static-mapping) and the engine deck modelling notebook), we should find the ratio $\overline{q}/q_\mathrm{TO}$ effectively weights the contributions of aerodynamic and rolling resistance penalties.
From inspection $\overline{q}/q_\mathrm{TO}\in [0, 1]$ to maximise aerodynamic or rolling resistance penalties, or:

$$
\frac{\overline{T}}{W}
= \frac{V^2_L}{2d_\mathrm{G}g}
+ \text{max} \left( \frac{C_D^\mathrm{TO}}{C_L^\mathrm{TO}}\,\,,\,\,\mu_\mathrm{R} \right)
$$

For a first-order approximation, either of the penalties are treated as constants - meaning there is no need to compute the dynamic pressure.

The proximity of the wings to the ground result in disturbances to the flow field around the aircraft, which have the effect of reducing the drag as a function of the *ground influence coefficient*, $\phi$.
The coefficient of induced drag *in ground effect* (IGE) is proportional to $C_{D_i}$ out of ground effect (OGE)

$$
C_{D_{i, \text{IGE}}}^{TO}
= \phi \cdot C_{D_{i, \text{OGE}}}^{TO}
\quad \text{with} \quad
\phi(h) \leq 1
$$

where $h$ is the height of the wings off the ground.

Also using the 

$$V_L\approx V_\mathrm{R(TAS)} = 1.1V_\mathrm{S1(TAS)}^\mathrm{TO} = 1.1\sqrt{\frac{2 W/S}{\rho C_\mathrm{Lmax}^\mathrm{TO}}}$$

approximation introduced earlier (note that we have $\rho$ in the equation this time, and not $\rho_0$, as we are dealing with TAS - this is because $V_\mathrm{L}$ comes from a purely kinematic calculation and it is therefore, technically, ground speed, which, with the zero wind assumption, is equal to TAS), the (average) thrust to weight ratio required for a ground roll of $d_\mathrm{G}$ can be written as:

$$
\frac{\overline{T}}{W} =
1.21\frac{W/S}{\rho C_\mathrm{Lmax}^\mathrm{TO}gd_\mathrm{G}}
+\text{max} \left( \frac{C_D^\mathrm{TO} \left(C_L^\mathrm{TO}, \phi\right)}{C_L^\mathrm{TO}}\,\,,\,\,\mu_\mathrm{R} \right)
$$

where $C_D^\mathrm{TO}$ is taken to be a function of the lift coefficient $C_L^\mathrm{TO}$ and ground influence coefficient $\phi$.

Broadly speaking, this equation breaks the required thrust to weight ratio down into three components.

The first is an inertial component -- the $\overline{T}/W$ required to accelerate the mass of the aeroplane to take-off speed. There is an element of aerodynamics here too, as the speed we need to achieve for lift-off depends on the maximum lift coefficient $C_\mathrm{Lmax}^\mathrm{TO}$.
Further extending the high lift devices is the obvious way to increase this, but then that also increases drag (penalised by the next term), which is why high lift devices tend not be used to their full capacity at take-off (the full capacity is usually set by the landing constraint).

The second component deals with most of the aerodynamics -- this rewards an aerodynamically efficient airframe; the higher the drag to lift ratio, the more thrust the propulsion system will have to supply.

The third term is driven by the rolling resistance, accounting for the thrust 'wasted' in overcoming the rolling resistance of the wheels. For CS-23/CS-25 scale aircraft values of $\mu_\mathrm{R}$ range from 0.03 (dry asphalt) to 0.08 (wet grass); higher values may be used for small unmanned aircraft.

#### The small print

Before pressing this model into service, we need to clarify some of its notations, limitations and assumptions.

First, the velocities. The calculations above assume zero wind. This means that we can just talk about 'speed' without specifying whether that mean *ground speed* in the inertial terms and *true airspeed* in the aerodynamic terms.

Next, the aerodynamic coefficients. $C_\mathrm{D}^\mathrm{TO}$ and $C_\mathrm{L}^\mathrm{TO}$ are the drag and lift coefficient values for the aircraft rolling along the runway, in take-off configuration (landing gear and take-off flaps down), wing in ground effect.

Further, the calculations set out above are based on the approximation that at the speed $V_\mathrm{L}$ in this take-off configuration and with wheels still on the ground, the net lift generated by the aircraft is equal to its weight. A moment later, when rotation occurs, everything will change as the angle of attack increases: the aircraft will generate more drag, but also more lift; as the lift exceeds the weight of the aircraft, a positive vertical acceleration will result as the wheels leave the ground.

$S_\mathrm{ref}$ is the reference area of the wing, with respect to which $C_\mathrm{D}^\mathrm{TO}$ and $C_\mathrm{L}^\mathrm{TO}$ are defined. In general, this is the planform (projected) area of the wing, though there are subtle differences in how this is calculated. The source of the variety of approaches here is that, on most aircraft, the low pressure areas above the two wings join together over the fuselage, so it makes sense to include into the area accounting some type of virtual extension of the wings up to the centreline of the aircraft. At this early stage in the design process we shall consider a simple trapeze planform, but it is important to be aware of the different methods used by different aircraft manufacturers when using their wing area (or lift/drag coefficient) data. For example, the wing area of Boeing aircraft is generally quoted using the *Wimpress method* (named after former chief aerodynamicist John K. Wimpress).

Next, a comment on the weight. $W$ in the equations above is clearly a *force*, but the fact that 'take-off weight' is almost always quoted in units of mass (typically kg, lbs, or tonnes) may confuse matters. Such numbers have to be converted into units of force when using these equations.

## Climb
[Back to Contents ⬆️](#Contents)

<img src="../img/climb.png" width="45%" align="left">
Simplified sketch of forces acting on an aircraft in the climb (background image of a Lockheed Martin F-22 Raptor courtesy of the US Air Force).

### Fundamental concepts

- **Climb angle** $\theta$ - the angle between the horizontal and the tangent to the flightpath of (the centre of gravity of) the aircraft. Positive for climbing flight, negative for descents.

- **Climb gradient** - usually expressed in percentage = $100\times\tan\theta$

- **Angle of attack** ($\alpha$) - if the airframe reference we use is the wing chord, then this is the angle of attack of the wing (if a fuselage reference is used, then the setting angle of the wing must be taken into account).

- **Rate of climb** (RoC) is the vertical component of the true airspeed $V$, so RoC = $V\sin\theta$ = $\frac{\mathrm{d}h}{\mathrm{d}t}$.

- **The inertial effect of density lapse** If the aircraft climbs at a constant indicated airspeed (or constant calibrated airspeed, in practice), its true airspeed (which is what matters from the point of view of the acceleration term in Newton's second law) will increase as the ambient density drops along its ascending path [example from *Jet Transport Performance Models (Boeing)*: over the first 1000 feet of a climb at sea level 150KIAS, the true airspeed will have increased by 4KTAS]. The resulting inertial term can be written, in the direction of the airspeed vector V, as:
$$
ma = \frac{W}{g}\frac {\mathrm{d}V}  {\mathrm{d}t}
\\
$$
where $m$ and $W$ are the mass and weight of the aircraft respectively.

- **The inertial effect of angle of climb variation** Along with the true airspeed, in the most general case, the angle of climb $\theta$ may vary during the climb too. In other words, the direction of the true velocity vector $\overrightarrow{V}$, along which we considered the inertial term above, may itself, be changing. This turns the trajectory into a curve. Approximating this curve as a circular arc, we can think of the force changing the climb gradient as a centripetal force $mr\omega^2$, where $\omega$ is the angular velocity, which is also the rate of change of the angle of the tangent to the trajectory, so $\omega = \mathrm{d}\theta / \mathrm{d}t$. Since the true airspeed $V=r\omega$, the centripetal force can be written as:
$$mV\omega = \frac{W}{g}V\frac{\mathrm{d}\theta}{\mathrm{d}t}$$

### T/W required for sustaining a given rate of climb

#### Perpendicular to the flight path

Projecting $F = ma$ onto the $z$ axis (perpendicular to the flight path):

$$\underbrace{L-W\cos{\theta}+T\sin\alpha}_{\textbf{z-components of lift, weight and thrust}} = \overbrace{-\frac{W}{g}V\frac {\mathrm{d}\theta}{\mathrm{d}t}}^{\textbf{mass}\,\times\,\textbf{acc. due to climb angle change}} $$

A possible shortcut at this stage, if a 'clean' analytical solution is desired, is that, since for the angles of attack $\alpha$ typical of a steady state climb $\sin\alpha <0.1$, we drop the $\sin$ term altogether:

$$L-W\cos{\theta} = -\frac{W}{g}V\frac {\mathrm{d}\theta}{\mathrm{d}t}$$

Further, if we assume that the time derivative of the climb angle is negligible (so the right hand side of the equation above is zero), the lift equation can be written as:

$$L = W\cos\theta = qSC_\mathrm{L}.$$

#### Along the flight path

Projecting the climb forces onto the $x$ (trajectory tangent) axis:

$$-D-W\sin\theta+T=\overbrace{\frac{W}{g}\frac{\mathrm{d}V}{\mathrm{d}t}}^{\textbf{mass}\,\times\,\textbf{acc. due to true airspeed change}}$$

Dividing each side by $W$ we obtain:

$$\frac{1}{W}\left( -D-W\sin\theta+T  \right)=\frac{1}{g}\frac{\mathrm{d}V}{\mathrm{d}t}$$

We can express the sine of the climb angle from here:

$$\sin\theta = \frac{T}{W}-\frac{D}{W} - \frac{1}{g}\frac{\mathrm{d}V}{\mathrm{d}t}.$$

We rewrite the vertical acceleration term, using the chain rule, as: 

$$\frac{\mathrm{d}V}{\mathrm{d}t} = \frac{\mathrm{d}V}{\mathrm{d}h} \frac{\mathrm{d}h}{\mathrm{d}t}$$

where $h$ is the geopotential altitude. The climb rate $\frac{\mathrm{d}h}{\mathrm{d}t} = V\sin\theta$ and thus

$$\sin\theta = \frac{T}{W}-\frac{D}{W} - \frac{1}{g}\frac{\mathrm{d}V}{\mathrm{d}h}V\sin\theta.$$

Rearranging, we obtain:

$$\sin\theta\left(1+ \frac{V}{g}\frac{\mathrm{d}V}{\mathrm{d}h}\right) = \frac{T}{W}-\frac{D}{W} $$

Replace the drag force with a drag coefficient:

$$\sin\theta\left(1+ \frac{V}{g}\frac{\mathrm{d}V}{\mathrm{d}h}\right) = \frac{T}{W}-\frac{qSC_D}{W} $$

Assuming that $C_D = f(C_L)$ and recalling our earlier definition of lift, we can replace $C_D$ with:

$$
\begin{eqnarray}
\sin\theta\left(1+ \frac{V}{g}\frac{\mathrm{d}V}{\mathrm{d}h}\right)
&=& \frac{T}{W}-\frac{qS \cdot f(C_L)}{W} \\
&=& \frac{T}{W}-\frac{q \cdot f\left( \frac{W}{S}\frac{\cos \theta}{q} \right)}{W/S} 
\end{eqnarray}
$$

The second term on the left-hand side of the equation is sometimes referred to as the *acceleration factor*:

$$K_\mathrm{a} = 1+ \frac{V}{g}\frac{\mathrm{d}V}{\mathrm{d}h}$$

Thus:

$$K_\mathrm{a}\sin\theta = \frac{T}{W}-\frac{q \cdot f\left( \frac{W}{S}\frac{\cos \theta}{q} \right)}{W/S} $$

and from here the thrust to weight ratio required can be expressed as:

$$\frac{T}{W} = \frac{q \cdot f\left( \frac{W}{S}\frac{\cos \theta}{q} \right)}{W/S}  + K_\mathrm{a}\sin\theta$$

or, in terms of the rate of climb $\text{RoC}$ (= $V\sin\theta$):

$$\frac{T}{W} = \frac{q \cdot f\left( \frac{W}{S}\frac{\cos \theta}{q} \right)}{W/S}  + K_\mathrm{a}\frac{\mathrm{RoC}}{V}$$

For a rapid, first order estimate, we could assume a shallow climb, that is, $\cos\theta\approx1$ and $\mathrm{d}V/\mathrm{d}h\approx0$, so $K_a=1$, in which case:

$$\frac{T}{W} \approx \frac{q \cdot f\left( \frac{W/S}{q} \right)}{W/S} + \frac{\mathrm{RoC}}{V}$$

There are two tricky-to-estimate items here.
One is the function $C_D = f(C_L)$ (we'll take a look at this in the [next notebook](./)).
The other is $V$, the climb speed.
Technically this should be the best rate of climb speed $V_{\text{Y}}$, but at this point in the design process we don't really know that. For both we therefore need to look at historical data, if available, and use suitable error margins around our estimated values.
Also, for an initial climb constraint the typical sub-10,000 feet speed limit (introduced for air traffic management reasons) of 250 knots should be seen as an upper limit to $V$. 

## Cruise
[Back to Contents ⬆️](#Contents)

From the condition that for steady, level cruising flight thrust must equal drag:

$$T=D=\frac{1}{2}\rho V^2S C_\mathrm{D} = qS \cdot f\left( C_L \right)$$

where, as before, $f$ is the function that produces $C_D$ as a function of $C_L$.

Since lift must equal weight, we can extract the lift coefficient from the lift equation and write it in terms of the weight $W$ as $C_\mathrm{L}=2W/\rho V^2 S$. Inserting this into the above and rearranging yields:

$$\frac{T}{W} = \frac{q \cdot f\left( \frac{W/S}{q}\right)}{W/S}  $$

It may be noted that this is basically the rate of climb constraint with the rate of climb set to zero, though, of course, $q$ will be different - it will be the $q$ corresponding to the desired cruise speed.

Note also that the cruise speed (which plugs into the $q$ above) is a true airspeed (TAS), standing in as a surrogate for ground speed (on the zero wind assumption) - after all, when we specify a cruise speed requirement for an aeroplane, we are interested in how long it takes to get from A to B, not what number is displayed on its airspeed indicator.

## Sustained turn
[Back to Contents ⬆️](#Contents)

<img src="../img/turn.png" width="20%" align="left">

*Forces in a turn (background image of an Aerostar R40 Festival by A Sóbester).*

### T/W required for sustaining a load factor *n*

Writing the balance of forces on the basis of the diagram above:

$$
\begin{eqnarray}
T&-&D&=&0 \\
L\,\mathrm{cos}\phi\, &-&W&=&0 \\
L\,\mathrm{sin}\phi\, &-&\frac{W}{g}\frac{V^2}{R_\mathrm{turn}}&=&0
\end{eqnarray}
$$

Note: the $V$ in the equation above is *true airspeed* (TAS), as this is an inertial term and the inertia is, of course, entirely independent of ambient air density (we are assuming zero wind).

From the first equation, breaking the drag down in the usual way (parabolic drag assumption):

$$T = D = qSC_{\mathrm{D}} = qS \cdot f(C_L) = qS \cdot f\left( \frac{W}{S}\frac{\cos \phi}{q} \right)$$

We define the *load factor* $n$ as the ratio of the lift and the weight (n=1 in steady, level flight), so $L = nW = qSC_\mathrm{L}$. From the force balance above, $n=1/\mathrm{cos}\,\phi$.

$$\frac{T}{W}=\frac{q \cdot f\left( \frac{W}{S} \frac{n}{q}\right)}{W/S}$$

Some additional, useful relationships that can be derived from the above force balance...

$$R_\mathrm{turn} = \frac{WV^2}{gL\sin\phi} = \frac{V^2}{gn\sin\phi} = \frac{V^2}{g\tan\phi}$$ or $$R_\mathrm{turn} = \frac{V^2}{g\sqrt{n^2-1}} $$

Note: From the lift required from the wing to accomplish the turn $L = nW = qSC_\mathrm{L}$, we can calculate the required lift coefficient: $C_\mathrm{L} = n\,(W/S)\,/q$. Of course, increasing the lift coefficient in the turn is only feasible by increasing the angle of attack, thus getting closer to the clean $C_\mathrm{L}^\mathrm{max}$. This means that there comes a point (at the higher end of the W/S scale) where the wing will not be able to generate the required lift (it will have reached its critical alpha), regardless of the amount of excees thrust available. You thus need to not only compute the T/W for a range of wing loadings, but also check that the required lift coefficient does not exceed $C_\mathrm{L}^\mathrm{max}$.

## Service ceiling
[Back to Contents ⬆️](#Contents)

### The best rate of climb airspeed

The service ceiling of an aircraft is defined as the maximum altitude at which the aircraft is able to attain a specified minimal rate of climb - usually 100fpm (0.508m/s) -- when flying at best climbing speed. This means that we can simply use the rate of climb constraint formulation here too, with the caveat that the accuracy of the ceiling constraint calculation depends on our estimate of the best climb speed. 

For propeller aircraft Gudmundsson (page 833, Equation 18-27) offers the following estimate of the best rate of climb speed (in terms of true airspeed):

$$V_\mathrm{Y} = \sqrt{\frac{2}{\rho}\left(\frac{W}{S}\right)   \sqrt{ \frac{k}{3\,C_\mathrm{Dmin}} }   }$$

This is the true airspeed that yields the maximum excess thrust (that is, where the difference between power available and power required is the greatest).

Estimates can be obtained using `AircraftConcept.get_bestV_Y`, however, when applied to aircraft reverse engineered with this library, these can be significantly different to the published speeds in the pilot's operating handbook.
Note that the ADRpy method will produce a result that differs from the analytical method, as it includes extra parameters for higher fidelity drag modelling.
At this point of the design process, we have limited means to come up with better estimates - unless we can make guesses based on existing, similar designs.

### Adapting the rate of climb constraint

The service ceiling constraint is, essentially, a rate of climb constraint (with a set rate 
of climb of 0.5m/s), so its implementation (in the `AircraftConcept.constrain_servceil` method) is very similar to that of the climb rate constraint. It requires two additional `designbrief` entries: `servceil_m` (the required service ceiling in metres) and `secclimbspd_kias` (the best indicated airspeed for climb to the service ceiling).

# References
<!-- <a name="refs"></a> -->

[1]  Gudmundsson, S., "General Aviation Aircraft Design: Applied Methods and Procedures", Butterworth-Heinemann, 2013. 